# Predicting Zillow Prices
### Nick Lind

This notebook was created to serve as an illustrative guide to the data science workflow.

## Step 1: Framing the problem and looking at the bigger picture
1. Define the objective in business terms. What decision are you trying to inform? What problem are you trying to solve?
2. How will your solution be used? Who is your user?
3. What are the current solutions/workarounds (if any)?
4. What type of learning problem are we dealing with (supervised, unsupervised, etc.)?
5. How should performance be measured? Is the performance measure aligned with the business objective?
7. What would be the minimum performance needed to reach the business objective?
8. What are comparable problems? Can you reuse experience or tools?

## Step 2: Procure the data
1. List the data you need and how much you need.
2. Find and document where you can get that data.
3. Check how much space it will take.
4. Check legal obligations, and get authorization if necessary.
5. Get access authorizations.
6. Create a workspace (with enough storage space).
7. Get the data.
8. Double-check for biases.
8. Convert the data to a format you can easily manipulate (without changing the data itself).
9. Ensure sensitive information is deleted or protected (e.g., anonymized).
10. Check the size and type of data (time series, sample, geographical, etc.).

### Data Types
For most data science problems in Python, we'll use:
    * Lists
    * Series (NumPy)
    * Arrays (NumPy)
    * DataFrames (Pandas)

### Import Data & Packages
First, we import our dataset as well as the relevant packages we'll need for our analysis.

In [ ]:
# Import packages
import numpy as np
import pandas as pd
from numpy import loadtxt
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense

import lightgbm as lgb
import seaborn as sns
import matplotlib.pyplot as plt
import ggplot as ggplot
color = sns.color_palette()
import folium


# Set parameters
path = "C:/Users/nthor/Documents/GitHub/Predicting_Property_Value_Zillow/data/"
loc_properties = "properties_2016.csv"
loc_train = "train_2016_v2.csv"
trainingsize = .8

In [ ]:
# Import files
transactions = pd.read_csv(path + loc_train, delimiter=',', parse_dates=["transactiondate"])
properties = pd.read_csv(path + loc_properties, delimiter=',')

Let's take a look at our two datasets.

In [ ]:
# Examine properties set
properties.head()

In [ ]:
# Examine list of sold homes
transactions.head()

Finally, let's join our datasets together before digging-in to our analysis.

Densely populated at zero, with slightly more spread at either end of the x-axis; studios and large mansions may be harder to accurately predict sales price.

In [ ]:
plt.figure(figsize=(6,6))
sns.jointplot(x=merged.finishedsquarefeet12.values, y=merged.bedroomcnt.values, size=10, color=color[4])
plt.ylabel('Bedroom Count', fontsize=12)
plt.xlabel('Finished Square Feet 12', fontsize=12)
plt.title("Finished square feet 12 Vs Bedroom Count", fontsize=15)
plt.show()

Many of our exogeneous variables are likely to be correlated (as illustrated above), which we'll have to keep in mind as we select our features.

### Additional steps to consider
1. Some inputs should be treated as categorical variables instead of integers
2. Identify potential distributions for parametric modeling (Gaussian, etc.)
3. Decompose dates / times into the appropriate format(s)

## Step 4: Clean the data

1. Drop useless features / attributes.
2. Fill-in NA's.
3. Drop outliers that you can't explain.
4. Create binary flags.
5. Convert datatypes where necessary.

In [ ]:
# split data into X and y while filling nulls with 0 and dropping a few categorical variables that we may want to add in later
X = merged.drop(['parcelid', 'logerror', 'transactiondate', 'transaction_month', 'accurate_price', 'propertycountylandusecode', 'propertyzoningdesc'], axis=1).fillna(0) # dropped for simplicity; generally you wouldn't want to drop variables without a strong rationale
y = merged['logerror']

# convert taxdelinquencyflag 'Y's into 1's
X.iloc[X[X['taxdelinquencyflag'] == 'Y'].index, X.columns.get_loc('taxdelinquencyflag')] = 1

# convert X to float
X = X.apply(pd.to_numeric)

## Step 5: Engineer features for modeling
* Discretize continuous features.
* Decompose features (e.g., categorical, date/time, etc.).
* Add promising transformations of features (e.g., log(x), sqrt(x), x^2, etc.).
* Aggregate features into promising new features.
* Brainstorm potentially predictive drivers and see what other data you can scrape / procure to improve your results.

For the purposes of this demo, we'll create a few simple features using abstractions of data we already have. In the real-world, you'll also want to 

In [ ]:
#life of property
X['N-life'] = 2018 - X['yearbuilt']

#Does property have a garage, pool or hot tub and AC?
X['N-GarPoolAC'] = ((X['garagecarcnt']>0) & (X['pooltypeid10']>0) & (X['airconditioningtypeid']!=5))*1 

X["N-location"] = X["latitude"] + X["longitude"]
X["N-location-2"] = X["latitude"]*X["longitude"]
X["N-location-2round"] = X["N-location-2"].round(-4)

X["N-latitude-round"] = X["latitude"].round(-4)
X["N-longitude-round"] = X["longitude"].round(-4)

#Number of properties in the zip
zip_count = X['regionidzip'].value_counts().to_dict()
X['N-zip_count'] = X['regionidzip'].map(zip_count)

#Number of properties in the city
city_count = X['regionidcity'].value_counts().to_dict()
X['N-city_count'] = X['regionidcity'].map(city_count)

#Number of properties in the city
region_count = X['regionidcounty'].value_counts().to_dict()
X['N-county_count'] = X['regionidcounty'].map(city_count)

TODO: Create new features after exploring initial Random Forest results.

## Step 6: Pre-process data for modeling
* Split our dataset into test and training sets so we can test how well our model generalizes the future
* Standardize / normalize our features to improve processing time and dampen the effect of outliers
    * Standardization <- (x - mean)/st.dev
        * Rescales features so that they'll have the properties of the standard normal distribution
        * Data's mean becomes 0 and st.dev. Becomes 1
        * Better performance
        * Mostly used for Gaussian distributions
    * Normalization <- (X - Xmin)/(Xmax - Xmin)
        * Also called Min-Max Scaling
        * Causes you to lose some information in the data, especially concerning outliers
        * Forces values to fit between the range of 0 and 1
        * Good when you do not know the distribution of your data or you know it's not Gaussian

In [ ]:
# Merge datasets
merged = pd.merge(properties, transactions, on=['parcelid','parcelid'])
# Fix latitude and longitude coordinates
merged.latitude = merged.latitude / 1e6
merged.longitude = merged.longitude / 1e6

### Examine Biases

Before moving forward, we should address the following questions:
    1. Did we introduce bias into the dataset during sampling?
    2. Do we have enough observations to establish causal inference?
    3. Do we have time-series data extending back far enough to detect regression to the mean?

## Step 3: Explore the data
3. Study each attribute and its characteristics:
    * Name
    * Type (categorical, int/float, bounded/unbounded, text, structured, etc.)
    * Percent of missing values
    * Noisiness and type of noise (stochastic, outliers, rounding errors, etc.)
    * Possibly useful for the task?
    * Type of distribution (Gaussian, uniform, logarithmic, etc.)
4. For supervised learning tasks, identify the target attribute(s).
5. Visualize the data.
6. Study the correlations between attributes.
7. Study how you would solve the problem manually.
8. Identify the promising transformations you may want to apply.
9. Identify extra data that would be useful (go back to “Get the Data”).
10. Document what you have learned.

### Missing Values

We notice a few missing values in our .head() calls; let's examine the percent of values in each column that are null.

In [ ]:
from sklearn.decomposition import PCA

# Extract features
pca = PCA(n_components=5)
fit = pca.fit(X_s)
# Summarize components
print("Explained Variance: ", fit.explained_variance_ratio_)

In [ ]:
# Visualize results
pca_viz = PCA().fit(X_s);

with plt.style.context('seaborn-whitegrid'):
    plt.semilogy(pca_viz.explained_variance_ratio_, '--o');
    plt.semilogy(pca_viz.explained_variance_ratio_.cumsum(), '--o');
    plt.show()

We see a sharp drop-off in explained variance around the 45th component.

In [ ]:
# Recover weights of variables
ident = np.identity(X_train.shape[1])
coef = pd.DataFrame(pca.transform(ident), columns = ['PC-1', 'PC-2', 'PC-3', 'PC-4', 'PC-5'], index = X.columns)

# Print results (first five rows)
coef.sort_values(by = 'PC-1', ascending = True, kind = 'mergesort').head()

### Regularization
* L1: technique for feature selection by encouraging sparsity (sum of square of the weights)
* L2: penalize large individual weights (sum of weights)
![image.png](http://www.chioka.in/wp-content/uploads/2013/12/L1-vs-L2-properties-regularization.png)

More detailed explanation [here](http://www.chioka.in/differences-between-l1-and-l2-as-loss-function-and-regularization/). Our take-away: advanced, tree-based algorithms use L1 regularization to select features.

### Tree-Based Feature Importance (LightGBM)
Quick and dirty way to identify features before we build a cross-validated model. Here, variable importance is based on the number of times a variable is selected for splitting, weighted by the squared improvement to the model as a result of each split, and averaged over all trees. [Elith et al. 2008]

In [ ]:
# Convert dataset
d_train = lgb.Dataset(X, label=y.ravel())

# Set parameters
params = {}
params['max_bin'] = 10
params['learning_rate'] = 0.0021 # shrinkage_rate
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'l1'          # or 'mae'
params['sub_feature'] = 0.5      # feature_fraction -- OK, back to .5, but maybe later increase this
params['bagging_fraction'] = 0.85 # sub_row
params['bagging_freq'] = 40
params['num_leaves'] = 512        # num_leaf
params['min_data'] = 500         # min_data_in_leaf
params['min_hessian'] = 0.05     # min_sum_hessian_in_leaf
params['verbose'] = 5
params['feature_fraction_seed'] = 2
params['bagging_seed'] = 3

# Train model
# Train model
clf = lgb.train(params, 
               d_train,
               500)

# Plot importance
lgb.plot_importance(clf)
plt.show()

## Step 8: Build and test multiple models
1. If the data is huge, you may want to sample smaller training sets so you can train many different models in a reasonable time (be aware that this penalizes complex models such as large neural nets or Random Forests).
2. Train many quick and dirty models from different categories (e.g., linear, naive Bayes, SVM, Random Forests, neural net, etc.) using standard parameters.
3. Measure and compare their performance.
    * For each model, use N-fold cross-validation and compute the mean and standard deviation of the performance measure on the N folds.
4. Analyze the most significant variables for each algorithm.
5. Analyze the types of errors the models make.
    * What data would a human have used to avoid these errors?
6. Have a quick round of feature selection and engineering.
7. Have one or two more quick iterations of the five previous steps.
8. Short-list the top three to five most promising models, preferring models that make different types of errors.
9. Cross-validate each model (a good tutorial for cross-validating neural nets can be found [here](https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/))
10. Tune our parameters to improve our CV scores. Note that you should not tune your parameters using your test set; doing so will leak information to your model that will reduce it's ability to deliver accurate predictions.

### Introduction to Decision Trees, Random Forests, and Gradient Boosted Machines (GBMs)

Key concepts:
    * Decision trees
    * Bagging (ensembling multiple trees together to reduce variance)
    * Random Forests (randomly selecting features out of your feature set)
    * Boosting (using weak learners to learn from misclassified training samples and reduce bias)
    
Helpful visuals [here](http://xgboost.readthedocs.io/en/latest/model.html).

### LightGBM

Recently released into the open source community by Microsoft, LightGBM is a fast, distributed, high performance gradient boosting (GBDT, GBRT, GBM or MART) framework. Here, we cross-validate our previously-trained model and generate predictions.

In [ ]:
# Examine outliers
merged.logerror.describe()

We have a few major outliers on either end of the spectrum. In general, the Zestimate seems to overshoot the actual sales price.

### Time Variables
Next, we explore how our transactions vary across time.

In [ ]:
# Create month variable and accurate prediction indicator
merged['transaction_month'] = merged['transactiondate'].dt.month
merged['accurate_price'] = '0'
merged['accurate_price'].loc[(merged.logerror.values == 0)] == '1'



# Plot results
cnt_srs = merged.transaction_month.value_counts()
plt.figure(figsize=(12,6))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8, color=color[3])
plt.xticks(rotation='vertical')
plt.xlabel('Month of transaction', fontsize=12)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.show()

Unsurprisingly, most real estate activity occurs during the spring and summer (when it's most convenient for the average American to move / relocate.

### Geographic Area

In [ ]:
plt.figure(figsize=(12,12))
sns.jointplot(x=merged.latitude.values, y=merged.longitude.values, size=10)
plt.ylabel('Longitude', fontsize=12)
plt.xlabel('Latitude', fontsize=12)
plt.show()

Sales made across three regions in California. Any pattern by region?

In [ ]:
# Filter outliers > 8 std. dev. away and create array for plotting
filtered_pred = merged[abs(merged.logerror.values) > 8*np.std(merged.logerror.values)]
lat_long = np.transpose(np.array([filtered_pred.latitude, filtered_pred.longitude]))
lat = np.array([filtered_pred.latitude])
long = np.array([filtered_pred.longitude])

from ggplot import * 
ggplot(aes(x='latitude', y='longitude', color='logerror'), data= filtered_pred) + \
    geom_point() + \
    scale_color_gradient(low = 'aqua', high = 'black')

In [ ]:
# Take average for folium map
avg_lat = sum(merged.latitude.values)/len(merged.latitude.values)
avg_long = sum(merged.longitude.values)/len(merged.longitude.values)

# Plot folium map
map_1 = folium.Map(location=[avg_lat, avg_long], zoom_start = 8)

for i in range(0, len(filtered_pred)):
    folium.CircleMarker([filtered_pred.latitude.iloc[i], filtered_pred.longitude.iloc[i]],
                        radius=5,
                        color='#3186cc',
                        fill_color='#3186cc',).add_to(map_1)
map_1

There does seem to be some grouping in our errors, paticularly around Huntington Park, Compton, and Mission Viejo. We may be able to improve our predictions by using clustering / dimensionality reduction techniques to generate new features.

### Geospatial Clustering

In [ ]:
# Cluster by coordinates
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.cluster import DBSCAN
from geopy.distance import vincenty

dbscan = DBSCAN(eps=5/6371., min_samples=5, algorithm='ball_tree', metric='haversine') # 50 meters
dbscan.fit(np.radians(lat_long))

cluster_labels = dbscan.labels_
num_clusters = len(set(cluster_labels))
clusters = pd.Series([lat_long[cluster_labels == n] for n in range(num_clusters)])
print('Number of clusters: {}'.format(num_clusters))

When we filter our dataset to only include houses where *logerror* is outside two standard deviations, we seem to have 7 logical clusters using DBSCAN. We could use these clusters to classify new observations and create new features for our final model.

### Correlation Analysis

We use correlation analysis to explore the relationships between the various features in our dataset.

In [ ]:
# Define heat map
corrmat = merged.corr(method='spearman')
f, ax = plt.subplots(figsize=(8, 8))

# Draw the heatmap using seaborn
sns.heatmap(corrmat, vmax=1., square=True, annot = False)
plt.title("Important variables correlation map", fontsize=15)
plt.show()

Which x-variables are highly correlated with our predicted values?

In [ ]:
corrmat.logerror[corrmat.logerror.values > .025]

In plain English, the following variables appear to be highly correlated with mistaken Zestimates:
* House / basement size
* Number of bathrooms
* Number of bedrooms
* Number of fireplaces
* Number of garage spaces
* Number of stories
* Newer apartments
* Taxes paid

In short, Zillow's estimates seem to inappropriately price large / lavish / new homes. Basement size appears to be particularly correlated to logerror; perhaps the algorithm doesn't accurately value large basements?

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x="basementsqft", y="logerror", data = merged)
plt.ylabel('Log Error', fontsize=12)
plt.xlabel('Basement Sq. Ft.', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Log Error vs. Basement Size", fontsize=15)
plt.show()

We see a slight positive trend, with a greater degree of logerror variance near the end of the x-axis.

In [ ]:
merged['bedroomcnt'].loc[merged['bedroomcnt']>7] = 7 # Set maximum number of bedrooms to seven
plt.figure(figsize=(12,8))
sns.violinplot(x='bedroomcnt', y='logerror', data=merged)
plt.xlabel('Bedroom count', fontsize=12)
plt.ylabel('Log Error', fontsize=12)
plt.show()

In [ ]:
# Find percent of missing values in each column
merged.isnull().sum()/len(merged)*100

Visualized differently:

In [ ]:
missing_df = merged.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df = missing_df.loc[missing_df['missing_count']>0]
missing_df['missing_ratio'] = missing_df['missing_count'] / merged.shape[0]
missing_df = missing_df.sort_values(by='missing_count')

ind = np.arange(missing_df.shape[0])
width = 0.9
fig, ax = plt.subplots(figsize=(12,18))
rects = ax.barh(ind, missing_df.missing_ratio.values, color=color[2])
ax.set_yticks(ind)
ax.set_yticklabels(missing_df.column_name.values, rotation='horizontal')
ax.set_xlabel("Percentage of missing values")
ax.set_title("Percentage of missing values in each column")
plt.show()

Unsurpisingly, many of our more obscure features (e.g., *poolcnt*) are blank for the vast majority of homes. Thankfully, our dependent variable, *logerror*, is never null. 

How should we handle these nulls? Generally, we do one of the following:
* Use a model that can deal with nulls (random forest)
* Drop the nulls
* Predict the nulls

### Logerror

Let's start by examining the value that we're responsible for predicting.

In [ ]:
plt.figure(figsize=(12,8))
sns.distplot(merged.logerror.values, bins=50, kde=False)
plt.xlabel('logerror', fontsize=12)
plt.show()

In [ ]:
# Scale features
scaler = StandardScaler()
X = X.fillna(0)
X_s = scaler.fit_transform(X)
y_s = scaler.fit_transform(y.values.reshape(-1, 1))

# Split into test / training
X_train, X_test, y_train, y_test = train_test_split(X_s, y_s, test_size=0.2, random_state=7)



## Step 7: Select key features

**Curse of Dimensionality**: Many problems that do not exist in low-dimensional space arise in high-dimensional space. As we add additional features into our model, the number of observations necessary to cover the potential state landscape scales exponentially.

Potential solutions include:
* Variable selection via LASSO, ElasticNet, Recursive Feature Elimination, etc.
* Dimensionality reduction via PCA, autoencoders, etc.
* Importance scores from gradient-boosted tree algorithms
* Using commen sense and correlation tables to eliminate nonsensical features
* Using regularization to select features

For linear models, we'd prefer to use Recursive Feature Selection, ElasticNet, and LASSO to identify the variables we care about (all available through glmnet in R or sk-learn in Python). When we use a boosted tree approach, there are more effective, built-in tools we can use to identify the features we care about.

### Principle Component Analysis
In our final output table below, the variables with the highest weights should be considered most important.

In [ ]:
# Build model with 10-fold CV
d_train2 = lgb.Dataset(X_train, label=y_train.ravel())

clf_cv = lgb.cv(params, 
                d_train, 
                500, 
                nfold = 10, 
                early_stopping_rounds = 100,
                stratified = True)

# Generate predictions
lgbm_pred = clf.predict(X_test, num_iteration=clf.best_iteration)

### K-Fold Cross-Validation of XGBoost

XGBoost was originally created to detect the Higgs boson particle. Let's see how it performs for our dataset.